# Introdução ao SQL - Manipulando Esquemas

## Recapitulando

Comandos apresentados na aula passada.

## Objetivos

Ao final desta aula o aluno deverá conhecer:

1. Entender como criar um banco de dados;
1. As principais funções para criação de tabelas e objetos em um banco de dados;
1. Como manipular o esquema das tabelas;
1. Como utilizar o comando SELECT para criar uma nova tabela ou popular uma tabela já existente;
1. Como garantir a integridade do banco por meio das FKs;

# Criando um banco de dados e manipulando dados persistidos

Já sabemos como consultar informações em bancos de dados. Hoje vamos explorar como um banco de dados é criado e como inserir, atualizar e deletar dados.

CRUD é um acrônimo para:
- C: Create
- R: Read
- U: Update
- D: Delete

Em geral, as empresas possuem times totalmente dedicados à criação e manutenção de bancos de dados, que restringem a maioria (se não todos) dos comandos que aprenderemos hoje aos demais usuários.

Tais restrições são necessárias para que seja garantida a integridade e bom funcionamento do banco.

# DDL - Data Definition Language

**Exercício:**

Vamos criar juntos uma base de dados para o banco com duas tabelas `agencias` e `clientes`. Essas tabelas devem estar relacionadas.

Considere que um cliente deve ter conta em apenas uma agencia e uma agência deve ter muitos clientes.

A tabela `agências` deve conter código, nome, endereço e telefone da agência.

A tabela `clientes` deve conter um código do cliente, nome, data de nascimento, agência e conta.

## CREATE

Usado para criar um banco de dados, tabela e outros objetos em um banco de dados. Um guia sobre os tipos de campos pode ser encontrado [aqui](https://aprendendodotnet.wordpress.com/2012/03/10/sql-server-o-comando-create-table-e-tipos-de-dados/#:~:text=SQL%20Server%20%E2%80%93%20O%20Comando%20CREATE%20TABLE%20e,Comprimento%20Fi%20...%20%208%20more%20rows%20).

### Crie um novo banco de dados para nosso exemplo do banco.

    CREATE DATABASE banco WITH OWNER = 'seu usuario postgres';

### Crie uma tabela chamada `agencias`. 

**OBS:** Quais as condições em cada campo?

    CREATE TABLE agencias(
        agencia_id SERIAL PRIMARY KEY, -- condicão de unicidade + NOT NULL + índice
        nome varchar(255) NOT NULL, -- restricao de campo não nulo
        endreco varchar(255), 
        telefone varchar(255)
    );

### Crie uma tabela chamada `clientes`. Quais as condições em cada campo?

**OBS:** Como utitlizar a chave estrangeira?

    CREATE TABLE cliebtes(
        cliente_id SERIAL PRIMARY KEY, 
        nome VARCHAR(255), 
        rg TEXT UNIQUE, -- condição de unicidade, mas permite nulo se n especificar NOT NULL
        data_de_nascimento DATE, 
        agencia INT REFERENCES agencias(agencia_id), 
        conta INT NOT NULL,
        CHECK (conta > 0) -- valor inserido ou atualizado deve satisfazer condicao,
        CHECK (conta > agencia) -- apenas como exemplo, check pode envolver colunas da mesma tabela
    );

### Chave Primária e Chave Estrangeira

Considere o comando a seguir:

    CREATE TABLE cliente (
        -- Criando uma constraint de PK implicitamente na definição da coluna
        cliente_id SERIAL PRIMARY KEY,
        
        -- Criando uma constraint de FK implicitamente na definição da coluna
        agencia INT REFERENCES agencias(agencia_id)
            -- Estamos dizendo que quando uma tupla da tabela referenciada (agencia_composta) for deletada, 
            -- as referências a ela serão removidas automaticamente desta tabela.
            ON DELETE CASCADE -- Opções: RESTRICT, NO ACTION, SET NULL, SET DEFAULT
            ON UPDATE CASCADE
    );

No exemplo acima, criamos 2 restrições (constraints) implicitamente, uma PK e uma FK que referencia a coluna agencia_id da tabela agencias.

Em caso de chave primária/estrangeira composta, podemos utilizar a notação simplificada abaixo:

    Assumindo que existe uma tabela agência com PK composta formada por nome e cidade, poderíamos criar uma tabela cliente da seguinte forma:

    -- Quando não definimos uma lista de colunas, assume-se a PK da relação referenciada.
    CREATE TABLE cliente (
        nome text,
        cidade text,
        PRIMARY KEY (nome, cidade),
        nome_ag text,
        cidade_ag text,
        FOREIGN KEY (nome_ag, cidade_ag) references agencia -- esta tabela vai referenciar agencia
    );

**Obs: ON DELETE CASCADE só irá funcionar quando os registros da tabela referenciada forem deletados. Não funciona após um DROP TABLE CASCADE. Isso acontece, por que a CONSTRAINT é removida da tabela e, portanto, a regra ON DELETE é perdida.**

### Constraints

Constraints são regras que impomos a colunas de uma tabela. Por exemplo, chave primária, chave estrangeira, restrições de domínio, etc.

#### Exemplo 1 - Agencia e Clientes

    CREATE TABLE agencia_composta (
        nome VARCHAR(255) NOT NULL,
        cidade VARCHAR(255) NOT NULL,

        endereco VARCHAR(255),
        telefone VARCHAR(255),

        -- Criando uma constraint de PK composta explicitamente
        CONSTRAINT agencia_composta_pk PRIMARY KEY (nome, cidade)
    );

    CREATE TABLE cliente_composta (
        client_id SERIAL,
        nome_cliente VARCHAR(255) NOT NULL,
        data_nasc DATE,
        conta INT CONSTRAINT check_conta CHECK (conta > 0), -- Check criado como uma constraint

        nome_agencia VARCHAR(255) NOT NULL,
        cidade_agencia VARCHAR(255) NOT NULL,

        CONSTRAINT client_pk PRIMARY KEY (client_id),

        -- Criando uma constraint de FK utilizando CONSTRAINT explicitamente.
        CONSTRAINT client_agencia_fk_composta 
            FOREIGN KEY (nome_agencia, cidade_agencia)
                REFERENCES agencia_composta(nome, cidade)
                    ON DELETE CASCADE
    );

#### Exemplo 2 - Alunos e Turmas

    DROP TABLE IF EXISTS Aluno CASCADE;
    DROP TABLE IF EXISTS Turma CASCADE;
    DROP TABLE IF EXISTS Aluno_Turma;
    /*
    Aluno (m) - pertence - (n) Turma 
    */
    CREATE TABLE Aluno (
        Id_Aluno SERIAL PRIMARY KEY,
        Nome VARCHAR(255)
    );
    /*
    IdA1, Aluno1
    IdA2, Aluno2
    ...
    */
    CREATE TABLE Turma (
        Id_Turma SERIAL PRIMARY KEY,
        Nome VARCHAR(255)
    );
    /*
    IdT1, Turma1
    IdT2, Turma2
    ...
    */
    CREATE TABLE Aluno_Turma (
        Id_Aluno INT REFERENCES Aluno(Id_Aluno), 
        Id_Turma INT REFERENCES Turma(Id_Turma),
        CONSTRAINT aluno_turma_pk 
            PRIMARY KEY(Id_Aluno, Id_Turma)
    );

**Obs: A vantagem de se utilizar o comando CONSTRAINT é que podemos definir o nome da regra (e.g. turma_aluno_fk), o que facilita alterações posteriores via comando ALTER. Além disso, ele não tem limitações quanto ao número de colunas participantes de uma FK ou PK.**

### CREATE TABLE COM SELECT

Podemos criar uma tabela a partir de um comando SELECT.

    CREATE TABLE employees2
    AS 
    SELECT e.last_name, e.first_name
    FROM employees AS e;
    
Note que as colunas da tabela nova seguirão os mesmos tipos das colunas retornadas pelo comando SELECT.

Com esse comando, o esquema da tabela nova não terá regras de PK ou FK. Constraints podem ser adicionadas posteriormente via comandos ALTER, se necessário.

## ALTER
Usado para alterar a estrutura de tabelas ou outro objeto em um banco de dados.

### Corrija o nome da tabela cliebtes para clientes.

    ALTER TABLE cliebtes RENAME TO clientes;

### Corrija o nome da coluna endereço na tabela `agencias`.

    ALTER TABLE agencias
    RENAME COLUMN endreco to endereco;

### Altere o tipo da coluna nome na table clientes para TEXT.

    ALTER TABLE clientes
    ALTER COLUMN nome TYPE TEXT;

### Adicione a coluna bom pagador à tabela `cliente`.

    ALTER TABLE clientes
    ADD COLUMN pagador BOOLEAN;

### Adicione uma regra de chave estrangeira a tabela cliente.

Obs: A tabela cliente já posssui uma FK definida no comando CREATE. Vamos criar novamente utilizando ALTER apenas para demonstrar o comando.

Vamos adicionar a seguinte regra de FK: 

    FK: Cliente.agencia_id -> Agencia.agencia_id.

    ALTER TABLE cliente
    ADD CONSTRAINT teste_fk 
    FOREIGN KEY agencia_id REFERENCES agencia(agencia_id);

## DROP

Usado para apagar bancos de dados, tabelas e outros objetos.

### Delete a tabela `customers`.

    DROP TABLE agencias;
    DROP TABLE IF EXISTS agencias;  -- DROP TABLE [IF EXISTS] table_name; --> Entre colchetes: opcional!

### Delete a regra de chave estrangeira da tabela cliente.

    ALTER TABLE cliente DROP CONSTRAINT teste_fk;

### DROP TABLE E CASCADE

Deletar uma tabela que é referenciada por outras por meio de chaves estrangeiras pode tornar o banco de dados inconsistente.

Tentar rodar o comando DROP TABLE nessas condições gera um erro, por exemplo:

    ERROR:  cannot drop table turmas because other objects depend on it
    DETAIL:  constraint turma_aluno_id_turma_fkey on table turma_aluno depends on table turmas
    constraint teste_fk on table turma_aluno3 depends on table turmas
    HINT:  Use DROP ... CASCADE to drop the dependent objects too.

Uma alternativa é utilizar o comando CASCADE. 

Esse comando removerá todas as dependências (CONSTRAINTS de chave estrangeira) das tabelas que dependem daquela que será removida.

Note que esse comando não removerá registros, apenas removerá o link que existia entre outras tabelas e a tabela removida.

### Deletar a tabela turmas utilizando CASCADE.

Considere que existe uma tabela `turma_aluno` que possui uma FK apontando para a tabela turmas. 

    FK: turma_aluno.id_turma -> turmas.id_turma

Para remover a tabela turmas sem erros, devemos utilizar o comando CASCADE:

    DROP TABLE turmas CASCADE;

O resultado do comando acima fará com que a chave estrangeira de `turma_aluno` seja removida, juntamente com a tabela `turmas`.

Dessa forma, novas inserções na tabela `turma_aluno` não dependerão mais da tabela `turmas`.

# Índices

Exemplo encontrado [aqui](https://pt.wikibooks.org/wiki/PostgreSQL_Pr%C3%A1tico/DDL/%C3%8Dndices,_Tipos_de_Dados_e_Integridade_Referencial).

Uma tabela contendo os CEPs do Brasil, com 633.401 registros.

- sem indice

    SELECT * FROM cep_tabela WHERE cep = ‘60420440’;

    **Em 7691 ms**
   

- Pós adicionar um índice:

    ALTER TABLE cep_tabela ADD CONSTRAINT cep_pk PRIMARY KEY (cep);


    A mesma consulta anterior agora gasta apenas **10 ms**.

Como criar um indice?

    CREATE INDEX nomeindice ON tabela (lower (nomecampo));

Crie um indice na tabela `agencia`.

Query:

    CREATE INDEX agidindex ON agencias(agencia_id);
    
Resultado:

<img src="./images/Index.png" width="30%" height="30%"/>